In [1]:
import numpy as np
from copy import copy

In [19]:
def forced(b):
    # If all are losses, a loss is forced
    for r in action_space(b):
        if not winner(np.add(b, r)):
            return False
    return True

In [20]:
def winner(b):
    s = b.shape[0]
    for row in b:
        if np.sum(row) == s: return True
    for col in b.T:
        if np.sum(col) == s: return True
    if np.sum(b * np.identity(s)) >= s: return True
    if np.sum(b * np.flip(np.identity(s), 1)) >= s: return True
    # Otherwise game is not over
    return False

In [21]:
def action_space(b, remove_iso = True):
    remaining = []
    for i in range(b.shape[0]):
        for j in range(b.shape[1]):
            if b[i, j] == 0:
                z = np.zeros(b.shape, dtype = np.int32)
                z[i, j] = 1
                if remove_iso:
                    if len(remaining) > 0:
                        if array_in_list(z, remaining):
                            continue
                    for _ in range(4):
                        if not array_in_list(z, remaining):
                            remaining.append(z)
                        if not array_in_list(z.T, remaining):
                            remaining.append(z.T)
                        z = rotate(z)
                # Add move to returning list
                remaining.append(z)
                    
    return remaining

In [22]:
def least_forced_from_space(b, n = 0):
    if winner(b):
        return None
    forced_list = []
    for r in action_space(b, remove_iso = n < 2):
        a = b + r
        if winner(a): continue
        elif forced(a):
            forced_list.append(a)
        p = least_forced_from_space(a, n + 1)
        if type(p) != type(None): forced_list.append(p)
    if forced_list == []:
        return None
    return min(forced_list, key = lambda x: np.sum(x))      

In [25]:
q = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
])
%timeit least_forced_from_space(q)

34.8 s ± 3.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
def array_in_list(ar, l):
    for i in l:
        if np.array_equal(ar, i):
            return True
    return False

In [24]:
def rotate(x):
    """Rotates an array counter-clockwise"""
    n = np.zeros(x.shape, x.dtype)
    for i in range(x.shape[0]):
        n[:, i] = x[i][::-1]
    return n